In [1]:
using JLD2, FileIO
using Statistics, Core
#using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated


In [2]:
f = jldopen("../data/mixed_layer_simulation_Q-100_dTdz0.010_tau0.00_profiles.jld2","r+")

JLDFile C:\Users\janak\Dropbox (MIT)\6.S898\6S898-climate-parameterization\data\mixed_layer_simulation_Q-100_dTdz0.010_tau0.00_profiles.jld2 (read/write)
 ├─📂 parameters
 │  ├─🔢 density
 │  ├─🔢 specific_heat_capacity
 │  ├─🔢 viscosity
 │  ├─🔢 diffusivity
 │  ├─🔢 surface_cooling
 │  ├─🔢 temperature_gradient
 │  └─🔢 wind_stress
 ├─📂 boundary_conditions
 │  ├─📂 top
 │  │  ├─🔢 FT
 │  │  └─🔢 Fu
 │  └─📂 bottom
 │     └─🔢 dTdz
 ├─📂 grid
 │  ├─🔢 dim
 │  ├─🔢 Nx
 │  ├─🔢 Ny
 │  ├─🔢 Nz
 │  ├─🔢 Hx
 │  ├─🔢 Hy
 │  ├─🔢 Hz
 │  ├─🔢 Tx
 │  ├─🔢 Ty
 │  ├─🔢 Tz
 │  ├─🔢 Lx
 │  ├─🔢 Ly
 │  ├─🔢 Lz
 │  ├─🔢 Δx
 │  ├─🔢 Δy
 │  ├─🔢 Δz
 │  ├─🔢 Ax
 │  ├─🔢 Ay
 │  ├─🔢 Az
 │  ├─🔢 V
 │  ├─🔢 xC
 │  ├─🔢 yC
 │  ├─🔢 zC
 │  ├─🔢 xF
 │  ├─🔢 yF
 │  └─🔢 zF
 ├─📂 eos
 │  ├─🔢 ρ₀
 │  ├─🔢 βT
 │  ├─🔢 βS
 │  ├─🔢 βp
 │  ├─🔢 T₀
 │  ├─🔢 S₀
 │  ├─🔢 p₀
 │  ├─🔢 cᵥ
 │  └─🔢 αᵥ
 ├─📂 constants
 │  ├─🔢 Ω
 │  ├─🔢 f
 │  └─🔢 g
 ├─📂 closure
 │  ├─🔢 C
 │  ├─🔢 Cb
 │  ├─🔢 ν
 │  └─🔢 κ
 └─📂 timeseries
    ├─📂 t
    │  ├─🔢 0
    │  ├─🔢 161
    │  ├─🔢 262
    

In [3]:
grid_res = length(f["timeseries/T/0"])

256

In [4]:
convert_res = 32
@assert grid_res % convert_res == 0

In [41]:
function convert_to_low_res(convert_res, data)
    if length(data) == 1
        return data
    else
        @assert length(data) % convert_res == 0
        s = length(data)/convert_res
        temp_array = zeros(convert_res)
        for i in 1:convert_res
            t = data[Int((i-1)*s+1):Int(i*s)]
            temp_array[i] = Statistics.mean(t)
        end
        return temp_array
    end
end

convert_to_low_res (generic function with 1 method)

In [45]:
function change_res(convert_res, f)
    low_res_data = Dict()
    for k in keys(f["timeseries"])
        low_res_data[k] = Dict()
        for k_iter in keys(f["timeseries"][k])
            #println(k,k_iter)
            try
                low_res_data[k][k_iter] = convert_to_low_res(convert_res, f["timeseries"][k][k_iter])
            catch y
                println(k,k_iter)
                println(y)
                break
            end
        end
    end
    return low_res_data
end

change_res (generic function with 1 method)

In [47]:
low_res_data = change_res(convert_res, f)

Dict{Any,Any} with 14 entries:
  "nu"     => Dict{Any,Any}("91449"=>[6.20838e-5, 5.64133e-5, 5.57755e-5, 5.574…
  "wT"     => Dict{Any,Any}("91449"=>[1.81145e-5, 1.94502e-5, 1.7048e-5, 1.4743…
  "T"      => Dict{Any,Any}("91449"=>[19.7078, 19.7112, 19.7118, 19.712, 19.712…
  "vv"     => Dict{Any,Any}("91449"=>[5.38115e-5, 3.3279e-5, 2.75752e-5, 2.5702…
  "uv"     => Dict{Any,Any}("91449"=>[5.63531e-6, 4.15572e-6, 3.29716e-6, 3.509…
  "ww"     => Dict{Any,Any}("91449"=>[1.88269e-5, 3.96846e-5, 4.86272e-5, 5.249…
  "t"      => Dict{Any,Any}("91449"=>1.72201e5,"350218"=>5.38201e5,"115245"=>2.…
  "vw"     => Dict{Any,Any}("91449"=>[1.9586e-7, 7.5001e-7, 2.00187e-6, 2.22979…
  "v"      => Dict{Any,Any}("91449"=>[-0.000130044, -0.000366867, -0.000489088,…
  "kappaT" => Dict{Any,Any}("91449"=>[9.72789e-5, 8.38152e-5, 8.14056e-5, 8.073…
  "w"      => Dict{Any,Any}("91449"=>[-4.43369e-22, 2.47228e-20, 8.89385e-21, 2…
  "uw"     => Dict{Any,Any}("91449"=>[1.07861e-6, 2.20488e-6, 4.23463e-6, 5.04

In [49]:
jldopen("32xres_data.jld2","w+") do file
    file["timeseries"] = low_res_data
end

Dict{Any,Any} with 14 entries:
  "nu"     => Dict{Any,Any}("91449"=>[6.20838e-5, 5.64133e-5, 5.57755e-5, 5.574…
  "wT"     => Dict{Any,Any}("91449"=>[1.81145e-5, 1.94502e-5, 1.7048e-5, 1.4743…
  "T"      => Dict{Any,Any}("91449"=>[19.7078, 19.7112, 19.7118, 19.712, 19.712…
  "vv"     => Dict{Any,Any}("91449"=>[5.38115e-5, 3.3279e-5, 2.75752e-5, 2.5702…
  "uv"     => Dict{Any,Any}("91449"=>[5.63531e-6, 4.15572e-6, 3.29716e-6, 3.509…
  "ww"     => Dict{Any,Any}("91449"=>[1.88269e-5, 3.96846e-5, 4.86272e-5, 5.249…
  "t"      => Dict{Any,Any}("91449"=>1.72201e5,"350218"=>5.38201e5,"115245"=>2.…
  "vw"     => Dict{Any,Any}("91449"=>[1.9586e-7, 7.5001e-7, 2.00187e-6, 2.22979…
  "v"      => Dict{Any,Any}("91449"=>[-0.000130044, -0.000366867, -0.000489088,…
  "kappaT" => Dict{Any,Any}("91449"=>[9.72789e-5, 8.38152e-5, 8.14056e-5, 8.073…
  "w"      => Dict{Any,Any}("91449"=>[-4.43369e-22, 2.47228e-20, 8.89385e-21, 2…
  "uw"     => Dict{Any,Any}("91449"=>[1.07861e-6, 2.20488e-6, 4.23463e-6, 5.04